In [2]:
import pandas as pd
import numpy as np

In [5]:
data_dir='C://Users//asus//Desktop//Netology//Data//ml-latest-small'
#data_dir='C://Users//mrasskazov//Desktop//DS//Data//ml-latest-small'
tags=pd.read_csv(data_dir+'/tags.csv')
ratings=pd.read_csv(data_dir+'/ratings.csv')
movies=pd.read_csv(data_dir+'/movies.csv')

In [6]:
tags['movie_id']=tags['movieId'].astype('category').cat.codes.copy()
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy()
ratings["user_id"] = ratings["userId"].astype("category").cat.codes.copy()

In [7]:
movies_tags = movies[["movieId"]].drop_duplicates().join(
    tags,
    on="movieId",
    rsuffix="codes",
    lsuffix="tags",
    sort=True
).fillna("None")[['tag','movieId']]
movies_tags.shape

(9125, 2)

In [8]:
movies_tags_rat = ratings[["movieId", 'movie_id']].drop_duplicates().join(
    movies_tags,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies_tags",
    sort=True
).fillna("None")[["tag", 'movie_id']]
movies_tags_rat.shape

(9066, 2)

Сливаем все теги для каждого фильма в одну строчку

In [182]:
tags_comb=tags.groupby('movieId')['tag'].apply('|'.join).reset_index()

Создаем матрицу тэгов

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
tags_features=CountVectorizer().fit_transform(movies_tags_rat['tag'])

Добавляем movie_id

In [10]:
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_tags_rat.shape[0]),
    tags_features
])

Получаем матрицу схожести между фильмами по тэгам

In [16]:
from implicit.nearest_neighbours import CosineRecommender
model=CosineRecommender()
model.fit(features)

In [19]:
model.similar_items(4)

[(4, 1.0000000000000002),
 (548, 0.25819888974716115),
 (845, 0.23570226039551589)]

In [13]:
list(zip(tags.tag[tags['movie_id']==88],tags.tag[tags['movie_id']==529]))

[('not too thrilled', 'toplist11')]

In [14]:
tags[tags['movie_id']==299]

,userId,movieId,tag,timestamp,movie_id
966,547,7438,holes00s,1342850493,299


In [15]:
movies[movies['movieId']==7438]

,movieId,title,genres
5152,7438,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller


In [20]:
user_feat=features.T.tocsr()

In [ ]:
model.pre

In [167]:
last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["user_id"].max()

In [170]:
from scipy.sparse import csr_matrix

user_x_item = ratings[["user_id", "movie_id"]].as_matrix()
mean_rating = ratings["rating"].mean()
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] > mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

In [214]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(ratings, test_size=0.2 )

In [171]:
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

размер обучающей выборки: 41289
размер тестовой выборки: 10279


In [226]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

# обучаем абы какую модель
model = LightFM(loss="warp")
model.fit(
    train_data,
    num_threads=4
)

train_mrr = auc_score(model, train_data).mean()
test_mrr = auc_score(model, test_data).mean()
train_prc = precision_at_k(model, train_data, k=10).mean()
test_prc = precision_at_k(model, test_data, k=10).mean()
train_rec = recall_at_k(model, train_data, k=10).mean()
test_rec = recall_at_k(model, test_data, k=10).mean()
print('ROC AUC: train %.2f, test %.2f, PRC: train %.2f, test %.2f, REC: train %.2f, test %.2f.' % (train_mrr, test_mrr,train_prc, test_prc,train_rec, test_rec))

ROC AUC: train 0.91, test 0.89, PRC: train 0.28, test 0.07, REC: train 0.08, test 0.07.


In [235]:
def get_recs_model(dataset):
    recs = model.predict(
        user_ids=np.array(dataset["user_id"].astype(np.int32).tolist()),
        item_ids=np.array(dataset["movie_id"].astype(np.int32).tolist()),
        num_threads=4
    )
    return recs

In [237]:
train_recs_model = get_recs_model(train)

In [198]:
hybrid = LightFM(loss="warp")
hybrid.fit(
    train_data,
    num_threads=4,
    item_features=features
)

train_mrr = auc_score(hybrid, train_data, item_features=features).mean()
test_mrr = auc_score(hybrid, test_data, item_features=features).mean()
train_prc = precision_at_k(hybrid, train_data, item_features=features,k=10).mean()
test_prc = precision_at_k(hybrid, test_data, item_features=features,k=10).mean()
train_rec = recall_at_k(hybrid, train_data, item_features=features,k=10).mean()
test_rec = recall_at_k(hybrid, test_data, item_features=features,k=10).mean()
print('ROC AUC: train %.2f, test %.2f, PRC: train %.2f, test %.2f, REC: train %.2f, test %.2f.' % (train_mrr, test_mrr,train_prc, test_prc,train_rec, test_rec))

ROC AUC: train 0.69, test 0.68, PRC: train 0.25, test 0.06, REC: train 0.07, test 0.06.


In [239]:
def get_recs_hybrid(dataset, features):
    recs = hybrid.predict(
        user_ids=np.array(dataset["user_id"].astype(np.int32).tolist()),
        item_ids=np.array(dataset["movie_id"].astype(np.int32).tolist()),
        item_features=features,
        num_threads=4
    )
    return recs

In [240]:
train_recs = get_recs_hybrid(train, features)